In [1]:
import os
os.chdir('..')

In [2]:
import torch
import numpy as np
from omegaconf import OmegaConf
from src.scoring import PrecalculationModule

/home/tsa87/anaconda3/envs/tacogfn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
MODEL_PATH = 'model_weights/model.tar'
HEAD_PATH = 'model_weights/zinc_1000_dim_256.pth'

In [4]:
head = torch.load(HEAD_PATH)
score_threshold = head["absolute_score_threshold"]

cfg = OmegaConf.create(head['config'])
focus_threshold = cfg.THRESHOLD.FOCUS
box_threshold = cfg.THRESHOLD.BOX

In [5]:
# predictor = PrecalculationModule(MODEL_PATH, HEAD_PATH, 'cuda')

In [5]:
from src.pharmaconet.modeling import ModelingModule
module = ModelingModule('model_weights/model.tar', 'cuda',
                        focus_threshold=focus_threshold,
                        box_threshold=box_threshold,
                        score_threshold=score_threshold)

In [7]:
split_file = torch.load('dataset/pocket_to_ligands.pt')

In [8]:
for rec_id, lig_id in split_file.items():
    rec_path = os.path.join('dataset/crossdock', rec_id + '_rec.pdb')
    lig_path = os.path.join('dataset/crossdocked_pocket10', lig_id)
    
    p_model = module.run(rec_path, ref_ligand_path=lig_path)
    break

In [12]:
p_model.nodes[0].feature

array([ 3.00607085e-03,  7.81376660e-01,  5.09609938e+00,  1.82631940e-01,
        3.34946930e-01,  3.16515565e-01,  6.78936899e-01, -4.44812663e-02,
       -1.11003570e-01,  9.69555825e-02,  3.61912325e-02,  1.02629565e-01,
        1.27877101e-01,  1.11688578e+00, -2.14803405e-02, -8.74845609e-02,
        2.04536974e-01,  1.17669797e+00,  1.67102173e-01, -2.35504165e-01,
        1.76707447e-01,  1.81422067e+00, -2.21573249e-01,  1.60369262e-01,
        5.66082954e-01, -2.50028968e-01,  1.59415781e-01,  1.78260505e-01,
        7.57520199e-02,  8.21047798e-02,  2.51354027e+00,  1.18668504e-01,
        2.19667912e-01,  4.62436295e+00,  3.52643430e-01,  1.83771753e+00,
        2.80970156e-01,  1.69603586e+00,  4.79777575e+00,  1.60204634e-01,
        3.95592308e+00,  1.44217551e-01, -1.91319749e-01,  2.07374144e+00,
       -2.03853533e-01,  1.06751304e-02,  1.91873825e+00,  4.11036871e-02,
        5.62935174e-01,  2.11866689e+00,  2.00081015e+00, -2.58941531e-01,
        1.91227865e+00,  

In [9]:
# center: [ -5.1225715 -32.597176    6.1419644]

In [23]:
pocket_image2, tokens2 = predictor._PrecalculationModule__parse_protein(rec_path,  np.array([-5.1225715, -32.597176, 6.1419644]))

In [8]:
print(rec_id)
print(len(p_model.nodes))

4xe6_X
64


In [16]:
p2_model['token_features'].shape

torch.Size([45, 256])